In [1]:
import sys
from pyspark.sql import SparkSession, functions
from pyspark.sql.types import StringType

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
spark

In [4]:
import wget
wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-02.parquet')
wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-02.parquet')
wget.download('https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-02.parquet')

'fhv_tripdata_2019-02.parquet'

In [5]:
df_fhv=spark.read\
    .option('header', 'true')\
        .parquet('C:/Users/Bloodink/Documents/data_fellowship_8_IYKRA/Data-Fellowship8-Austin-Christian/assignment6/fhv_tripdata_2019-02.parquet')

df1=spark.read\
    .option('header', 'true')\
        .parquet('C:/Users/Bloodink/Documents/data_fellowship_8_IYKRA/Data-Fellowship8-Austin-Christian/assignment6/yellow_tripdata_2019-02.parquet')

df2=spark.read\
    .option('header', 'true')\
        .parquet('C:/Users/Bloodink/Documents/data_fellowship_8_IYKRA/Data-Fellowship8-Austin-Christian/assignment6/green_tripdata_2019-02.parquet')


In [6]:
df_fhv.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00037|2019-02-01 07:08:44|2019-02-01 07:23:35|       264.0|       265.0|   null|                B00037|
|              B00037|2019-02-01 07:27:51|2019-02-01 07:32:54|       264.0|       265.0|   null|                B00037|
|              B00037|2019-02-01 07:18:30|2019-02-01 07:25:45|       264.0|       265.0|   null|                B00037|
|              B00037|2019-02-01 07:43:15|2019-02-01 07:48:29|       264.0|       265.0|   null|                B00037|
|              B00037|2019-02-01 07:01:45|2019-02-01 07:09:13|       264.0|       265.0|   null|                B00037|
+--------------------+------------------

In [7]:
df1.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2019-02-01 07:59:04|  2019-02-01 08:07:27|            1.0|          2.1|       1.0|                 N|          48|         234|           1|        9.0|  0.5|    0.5|       2.

In [8]:
df2.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-02-01 07:10:19|  2019-02-01 07:21:43|                 N|       1.0|          92|         135|            1.0|         2.79|       11.0|  0.5|    0.

In [9]:
df_fhv.createOrReplaceTempView('fhv')
df1.createOrReplaceTempView('tripdata')
df2.createOrReplaceTempView('tripdata_green')

## Problem 1
How many taxi trips were there on February 15?

In [10]:
df1.where((df1.tpep_pickup_datetime >= '2019-02-15')& (df1.tpep_pickup_datetime <= '2019-02-16')).count()

268129

In [11]:
df2.where((df2.lpep_pickup_datetime >= '2019-02-15') & (df2.lpep_pickup_datetime <= '2019-02-16') ).count()

23924

## Problem 2
Find the longest trip for each day ?

In [12]:
df1.withColumn('tpep_pickup_datetime',functions.to_date(df1.tpep_pickup_datetime)).groupBy('tpep_pickup_datetime').agg({'trip_distance':'max'}).orderBy('tpep_pickup_datetime').show()

+--------------------+------------------+
|tpep_pickup_datetime|max(trip_distance)|
+--------------------+------------------+
|          2008-12-31|             19.61|
|          2009-01-01|             21.66|
|          2009-01-02|              9.15|
|          2019-01-20|               0.0|
|          2019-01-30|               0.0|
|          2019-01-31|             20.33|
|          2019-02-01|             701.5|
|          2019-02-02|             71.04|
|          2019-02-03|            130.53|
|          2019-02-04|              77.9|
|          2019-02-05|             87.99|
|          2019-02-06|             74.98|
|          2019-02-07|              64.6|
|          2019-02-08|             74.24|
|          2019-02-09|             85.31|
|          2019-02-10|              90.7|
|          2019-02-11|             75.24|
|          2019-02-12|             84.41|
|          2019-02-13|             86.64|
|          2019-02-14|             76.89|
+--------------------+------------

In [13]:
df2.withColumn('lpep_pickup_datetime',functions.to_date(df2.lpep_pickup_datetime)).groupBy('lpep_pickup_datetime').agg({'trip_distance':'max'}).orderBy('lpep_pickup_datetime').show()

+--------------------+------------------+
|lpep_pickup_datetime|max(trip_distance)|
+--------------------+------------------+
|          2009-01-01|              6.43|
|          2010-09-23|             15.79|
|          2019-02-01|             79.85|
|          2019-02-02|              41.5|
|          2019-02-03|             40.34|
|          2019-02-04|              64.4|
|          2019-02-05|             61.42|
|          2019-02-06|              35.6|
|          2019-02-07|              38.1|
|          2019-02-08|             43.43|
|          2019-02-09|             49.38|
|          2019-02-10|             36.25|
|          2019-02-11|             48.27|
|          2019-02-12|             48.95|
|          2019-02-13|             38.76|
|          2019-02-14|              51.2|
|          2019-02-15|             103.7|
|          2019-02-16|             50.94|
|          2019-02-17|             56.96|
|          2019-02-18|             44.86|
+--------------------+------------

## Problem 3
Find Top 5 Most frequent `dispatching_base_num` ?

In [14]:
spark.sql("SELECT dispatching_base_num, COUNT(dispatching_base_num) as count FROM fhv GROUP BY dispatching_base_num Order By count DESC").show()

+--------------------+-----+
|dispatching_base_num|count|
+--------------------+-----+
|              B01145|44044|
|              B00256|37898|
|              B00900|37255|
|              B00856|36450|
|              B00647|32230|
|             B01268 |30735|
|              B01061|27958|
|              B00937|27545|
|              B01626|27327|
|              B01667|26791|
|              B01239|26725|
|              B02437|25208|
|              B00887|23427|
|              B03060|20304|
|              B00821|19630|
|              B01087|19316|
|              B01899|18380|
|              B00310|18118|
|              B01338|17716|
|              B02550|17551|
+--------------------+-----+
only showing top 20 rows



## Problem 4
Find Top 5 Most common location pairs (`PUlocationID` and `DOlocationID`) ?

In [15]:
spark.sql("SELECT PUlocationID, DOlocationID, COUNT(*) as count FROM fhv GROUP BY PUlocationID, DOlocationID Order By count DESC").show()

+------------+------------+-------+
|PUlocationID|DOlocationID|  count|
+------------+------------+-------+
|       264.0|       265.0|1193389|
|       264.0|       264.0| 261787|
|       265.0|       265.0|  39351|
|       198.0|       198.0|   3048|
|        92.0|        92.0|   2827|
|         7.0|         7.0|   2531|
|        56.0|        56.0|   2351|
|       206.0|       206.0|   2218|
|       132.0|       265.0|   1967|
|       223.0|       223.0|   1962|
|       162.0|       265.0|   1575|
|       179.0|       179.0|   1570|
|       230.0|       265.0|   1553|
|       129.0|       129.0|   1480|
|       173.0|       173.0|   1402|
|       161.0|       265.0|   1265|
|        36.0|        36.0|   1250|
|       138.0|       265.0|   1185|
|       198.0|        36.0|   1126|
|        82.0|        82.0|   1096|
+------------+------------+-------+
only showing top 20 rows

